# CLIPSeq-Toolkit Analysis Workflow

    This notebook will go step by step on how to use the various scripts available in the lw-clipseq-toolkit. The
    notebook will use test data available within the test_data directory as an example. However, the notebook is
    designed so that you can configure it to run your dataset through the workflow.
    
    Directions on how to use a jupyter notebook: To run a cell block, hold [Shift] then press [Enter].

In [2]:
# Import modules to help execute commands
import os
import pandas as pd

## Step 1: Convert gff3 annotation file into BED format

Before peaks can be annotated, the annotation file needs to be converted to a BED format. Annotations for mouse and humans can be obtained from Gencode (https://www.gencodegenes.org/). For other species, annotations can be obtained from Ensembl (https://useast.ensembl.org/index.html). The annotation file should follow the gff3 format (http://gmod.org/wiki/GFF3).

In [20]:
# Set path variables for gff3 file
gff_path = 'test_data/annotation.test.gff3'

# Set desired path for output files
out_file = 'test_data/'

Running the conversion will take time to run. The following output files will be created; features.bed, 
    intron.bed, and annotation.bed.

In [21]:
%%bash -s "$gff_path" "$out_file"

# Run the gff2bed.py script to convert gff to BED
python gff2bed.py -gff $1 -out $2

Conversion of gff to bed was successful.
Finished creating intron.bed annotation file.
Conversion of gff3 to BED format was successful.


The annotation.bed file will be used to annotate the clip peak file.

## Step 2: Annotate CLIP peaks

The CLIP peaks will be annotated using the annotation.bed file created in the previous step. The CLIP peaks file should be in a BED6 format with each row corresponding to a CLIP peak. BED6, a tab-delimited file with columns in the following order; chrom, start, stop, name, score, strand. 

In [22]:
# Set path variables for gff_bed file and peak file
gff_bed_path = 'test_data/annotation.bed'
peak_path = 'test_data/peak.test.bed'

# Set desired path for output files
out_file = 'test_data/'

In [32]:
%%bash -s "$peak_path" "$gff_bed_path" "$out_file"

# Run the annotate_peaks.py script to to annotate the peaks
python annotate_peaks.py -pk $1 -gbed $2 -out $3

Number of intersection errors: 0
Finished annotating peaks.


Completion of the script results in the output file 'annotated_peaks.csv'. The file has 10 columns with each row corresponding to a peak. 

In [37]:
# View the annotated peaks
annot_peaks = pd.read_csv(out_file + '/annotated_peaks.csv')
annot_peaks[annot_peaks['feature'] == 'exon'].head(n=10)

,peak_id,gene_name,gene_id,gene_type,chr,start,stop,strand,feature,exon_type
78,peak_79,Aqp12,ENSMUSG00000045091.4,protein_coding,chr1,93012021,93012167,+,exon,three_prime_UTR;stop_codon;CDS
83,peak_84,Gm15668,ENSMUSG00000086204.1,lincRNA,chr1,67250556,67250627,+,exon,NaN
108,peak_109,Il1rl2,ENSMUSG00000070942.8,protein_coding,chr1,40363220,40363340,+,exon,CDS
139,peak_140,Coa5,ENSMUSG00000026112.7,protein_coding,chr1,37417651,37417745,+,exon,three_prime_UTR
148,peak_149,Enah,ENSMUSG00000022995.16,protein_coding,chr1,181896915,181896987,+,exon,three_prime_UTR
153,peak_154,Coa5,ENSMUSG00000026112.7,protein_coding,chr1,37417598,37417651,+,exon,three_prime_UTR
176,peak_177,Fam174a,ENSMUSG00000051185.9,protein_coding,chr1,95327525,95327607,-,exon,three_prime_UTR
180,peak_181,Fn1,ENSMUSG00000026193.15,protein_coding,chr1,71644952,71645047,-,exon,NaN
206,peak_207,Fam174a,ENSMUSG00000051185.9,protein_coding,chr1,95326846,95326896,-,exon,three_prime_UTR
215,peak_216,Tns1,ENSMUSG00000055322.15,protein_coding,chr1,73995225,73995317,-,exon,CDS


annotated_peaks.csv column explanation:

1. The 'feature' column signifies what feature type the peak overlaps with, possible features are exon, exon-intron, intron, and intergenic. 

2. For peaks that overlap on 'exon' or 'exon-intron' features, the exon_type column specifies the possible exon_type. The possible types are CDS, three_prime_UTR, five_prime_UTR, start_codon, stop_codon. If there are multiple entries in the exon_type field, it signifies that different transcripts of that gene have different annotations for that region.

### Get simple summary of annotated peaks

In [38]:
# Set path variables for gff_bed file and peak file
annot_peak_path = 'test_data/annotated_peaks.csv'

# Set desired path for output files
out_file = 'test_data/'

In [40]:
%%bash -s "$annot_peak_path" "$out_file"

# Run the annotate_peaks.py script to to annotate the peaks
python peak_summary.py -ap $1 -out $2

Summary complete!


In [46]:
# Path to summary file
summary_file = out_file + '/feature_count_summary.csv'

# View summary table
summary_tbl = pd.read_csv(summary_file)
feature_sum = summary_tbl['exon'] + summary_tbl['intron'] + summary_tbl['intergenic'] + summary_tbl['exon-intron']
print('intergenic + exon + intron + exon-intron: %i' % feature_sum)
print('Total number of peaks from annotated peaks: %i' % annot_peaks.shape[0])
summary_tbl

intergenic + exon + intron + exon-intron: 489
Total number of peaks from annotated peaks: 489


,intergenic,exon,intron,exon-intron,CDS,five_prime_UTR,three_prime_UTR
0,127,22,324,16,16,0,12


## Step 3: Determine Overlap of CLIP Peaks with Exons from rMATS Output

It is often desired to overlap the CLIP data with results obtained from additional analysis. For instance, for a splicing factor, it is logical to assess if CLIP peaks are present on exons that are differentially spliced in settings of overexpression or knockdown of the splicing factor. rMATS is a popular pipeline that is used to determine differential exon usage between two conditions (control vs experiment). This toolkit also provides scripts to intersect rMATS output with the annotated CLIP peaks.

### Step 3a: rMATS Output Filtering and Formatting (Mandatory)

To perform the overlap, the rMATS data must be formatted and consolidated to allow the script to execute properly. This is achieved by using the rMATS_filter.py script. This script filters out the rMATS output based on various thresholds as well as format the data for usage in the script.

<b>Note on rMATS output:</b>

The rMATS output consists of 5 splice event type; skipped exon (SE), alternative 5' splice site (A5SS), alternative 3' splice site (A3SS), mutually exclusive exon (MXE) and retained intron (RI). It also has estimations based on only junction read counts (JC) or junction + exon read counts (JCEC). For this example analysis, all exon type events estimated using junction + exon read counts will be used

#### Set Filtering Threshold

In [52]:
# Set p-value filter threshold. This will keep all events that have p-values that are < the value specified.
# If filtering is not desired by pvalue, set this to 1.
pval_filter = 0.15

# Set FDR filter threshold. This will keep all events that have FDR that are < the value specified.
# If filtering is not desired by FDR, set this to 1.
fdr_filter = 0.15

# Set IncLevelDiff filter threshold. This will keep all events that have IncLevelDiff that are > the value specified.
# IncLevelDiff refers to the change in percent spliced-in value for a given event between two conditions.
# If filtering is not desired by FDR, set this to 0.
diff_filter = 0.15

# Set count filter threshold. This will keep all events that have read counts > the value specified.
# If filtering is not desired based on counts, set this to 0.
count_filter = 20

#### Specific rMAT data and output paths

In [48]:
# For convenience, the script has a -d option which allows for filtering of all rMAT files within a directory.
# This option will be used in this example.
rmats_output = 'test_data/rmats_test_data/jcec/'

# Set output directory
filter_out = 'test_data/rmats_test_data/'

#### Run filtering

In [54]:
%%bash -s "$rmats_output" "$pval_filter" "$fdr_filter" "$diff_filter" "$count_filter" "$filter_out"
python rmats_filter.py -d $1 $2 $3 $4 $5 $6

Finished with test_data/rmats_test_data/jcec/SE.MATS.JCEC.txt
Finished with test_data/rmats_test_data/jcec/RI.MATS.JCEC.txt
Finished with test_data/rmats_test_data/jcec/MXE.MATS.JCEC.txt
Finished with test_data/rmats_test_data/jcec/A5SS.MATS.JCEC.txt
Finished with test_data/rmats_test_data/jcec/A3SS.MATS.JCEC.txt
 
Filtering of rMATS output complete!


The filtered output contains separate filtered files for each exon event type and an additional file with all events combined into a single file.

### Step 3b: Find intersection of CLIP peaks with Exons from rMATS Output

This can be achieved by using the peaks2rmats.py script. The script requires only two inputs; 1) filtered and formatted rMATS data and 2) the annotated peaks that was obtained in Step 2 of the workflow.

In [57]:
# Set path variables
rmats_file = 'test_data/rmats_test_data/all_events_filtered_Pval-lt-0.15_FDR-lt-0.15_Diff-gt-0.15_Count-gt-20.csv'
peaks_file = 'test_data/annotated_peaks.csv'
out_path = 'test_data/'

In [61]:
%%bash -s "$rmats_file" "$peaks_file" "$out_path"
python peaks2rmats.py -rf $1 -ap $2 -out $3

Summary complete!


In [5]:
# The name of the output file is 'rmats_with_clip.csv'. We can view it using pandas.
rmats_with_clip = pd.read_csv('test_data/rmats_with_clip.csv')
rmats_with_clip.head(n=10)

,ID,GeneID,Event,geneSymbol,PValue,FDR,IncLevelDifference,Region_type,Feature_type,exon,lower,upper,exon_count,lower_count,upper_count,total
0,467,ENSMUSG00000026150.14,SE,Mff,0.000000e+00,0.000000e+00,0.296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,472,ENSMUSG00000026150.14,SE,Mff,1.110223e-16,5.139122e-15,-0.310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,748,ENSMUSG00000038599.14,SE,Capn8,5.683193e-10,1.472101e-08,0.518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,ENSMUSG00000026634.16,SE,Angel2,3.700040e-11,1.081012e-09,0.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1529,ENSMUSG00000056763.16,SE,Cspp1,4.681000e-04,4.856173e-03,-0.249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1535,ENSMUSG00000056763.16,SE,Cspp1,1.049416e-11,3.218567e-10,-0.151,intron,NaN,NaN,peak_410,NaN,0.0,1.0,0.0,1.0
6,1536,ENSMUSG00000056763.16,SE,Cspp1,2.162423e-08,4.622037e-07,0.158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1538,ENSMUSG00000026307.12,SE,Scly,1.125553e-03,1.074518e-02,0.179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1953,ENSMUSG00000026289.15,SE,Atg16l1,0.000000e+00,0.000000e+00,0.254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3070,ENSMUSG00000040225.15,SE,Prrc2c,0.000000e+00,0.000000e+00,0.549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This data is not a great example, since very few peaks overlap with the differentially spliced exons.

### peaks2rmat.py Output Format

Note: Columns 'ID' to 'IncLevelDifference' are copied from the rMATS file.

<b>Region_type</b>; specifies what regions of the splice event are peaks found: exon, exon-intron, intron

<b>Feature_type</b>; for exonic binding, what region of mRNA are peaks localized within: 5'UTR, CDS, 3'UTR, etc

<b>exon, lower, and upper</b>; list the peak_id of binding peaks from the annotated_peaks file that localized within the exon, downstream of exon (lower), upstream of exon (upper)

<b>exon_count, lower_count, upper_count</b>; specifies the number of peaks localized within the specified region

<b>total</b>; the total number of peaks that intersect with that exon event